### dataset
1. IMDB
2. CNNNews
3. [PubMed](https://github.com/LIAAD/KeywordExtractor-Datasets/blob/master/datasets/PubMed.zip)

### preprocess
1. filter too frequent and less frequent words
2. stemming
3. document vector aggregation

### evaluation
1. F1
2. NDCG

In [1]:
import os
from collections import defaultdict
import math
import numpy as np 
import pandas as pd
import random
import re
import torch
import torch.nn as nn
from itertools import cycle
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.optim as optim
from tqdm.auto import tqdm

# Used to get the data
from sklearn.metrics import ndcg_score

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
nltk.download('stopwords')

import matplotlib.pyplot as plt 
import matplotlib
matplotlib.use('Agg')

seed = 33

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chrisliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preprocess config

In [2]:
config = {}

config["dataset"] = "CNN" # "IMDB" "CNN", "PubMed"
config["n_document"] = 500
config["normalize_word_embedding"] = False
config["min_word_freq_threshold"] = 20
config["topk_word_freq_threshold"] = 100
config["document_vector_agg_weight"] = 'IDF' # ['mean', 'IDF', 'uniform', 'gaussian', 'exponential', 'pmi']
config["document_vector_weight_normalize"] = True # weighted sum or mean, True for mean, False for sum 
config["select_topk_TFIDF"] = None # ignore
config["embedding_file"] = "../data/fasttext.en.100d.txt"#"../data/fasttext.en.100d.txt"#"../data/glove.6B.100d.txt" #doc2vecC.imdb.200d.txt
# config["embedding_file"] = "../data/400d.txt"
config["topk"] = [30, 40, 50, 60, 70]

In [3]:
def in_notebook():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' not in get_ipython().config:  # pragma: no cover
            return False
    except ImportError:
        return False
    return True

In [4]:
def load_word2emb(embedding_file):
    word2embedding = dict()
    word_dim = int(re.findall(r".(\d+)d", embedding_file)[0])

    with open(embedding_file, "r") as f:
        for line in tqdm(f):
            line = line.strip().split()
            word = line[0]
            embedding = list(map(float, line[1:]))
            if len(embedding) != word_dim:
                print('error', word)
                continue
            word2embedding[word] = np.array(embedding)

    print("Number of words:%d" % len(word2embedding))

    return word2embedding

word2embedding = load_word2emb(config["embedding_file"])

0it [00:00, ?it/s]

Number of words:2000000


In [5]:
def normalize_wordemb(word2embedding):
    # Every word emb should have norm 1
    
    word_emb = []
    word_list = []
    for word, emb in word2embedding.items():
        word_list.append(word)
        word_emb.append(emb)

    word_emb = np.array(word_emb)

    for i in range(len(word_emb)):
        norm = np.linalg.norm(word_emb[i])
        word_emb[i] = word_emb[i] / norm

    for word, emb in tqdm(zip(word_list, word_emb)):
        word2embedding[word] = emb
    return word2embedding

if config["normalize_word_embedding"]:
    normalize_wordemb(word2embedding)

In [6]:
class Vocabulary:
    def __init__(self, word2embedding, config):
        # The low frequency words will be assigned as <UNK> token
        self.itos = {0: "<UNK>"}
        self.stoi = {"<UNK>": 0}
        
        self.word2embedding = word2embedding
        self.config = config

        self.word_freq_in_corpus = defaultdict(int)
        self.IDF = {}
        self.ps = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
        
        self.word_dim = len(word2embedding['the'])
    def __len__(self):
        return len(self.itos)

    def tokenizer_eng(self, text):
        text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
        text = text.strip().split()
        
        return [self.ps.stem(w) for w in text if w.lower() not in self.stop_words]
    
    def read_raw(self):        
        if self.config["dataset"] == 'IMDB':
            data_file_path = '../data/IMDB.txt'
        elif self.config["dataset"] == 'CNN':
            data_file_path = '../data/CNN.txt'
        elif self.config["dataset"] == 'PubMed':
            data_file_path = '../data/PubMed.txt'
        
        # raw documents
        self.raw_documents = []
        with open(data_file_path,'r',encoding='utf-8') as f:
            for line in tqdm(f, desc="Loading documents"):
                self.raw_documents.append(line.strip("\n"))
                
        return self.raw_documents
    
    def build_vocabulary(self):
        sentence_list = self.raw_documents
        
        self.doc_freq = defaultdict(int) # # of document a word appear
        self.document_num = len(sentence_list)
        self.word_vectors = [[0]*self.word_dim] # unknown word emb
        
        for sentence in tqdm(sentence_list, desc="Preprocessing documents"):
            # for doc_freq
            document_words = set()
            
            for word in self.tokenizer_eng(sentence):
                # pass unknown word
                if word not in self.word2embedding:
                    continue
                    
                # calculate word freq
                self.word_freq_in_corpus[word] += 1
                document_words.add(word)
                
            for word in document_words:
                self.doc_freq[word] += 1
        
        # calculate IDF
        print('doc num', self.document_num)
        for word, freq in self.doc_freq.items():
            self.IDF[word] = math.log(self.document_num / (freq+1))
        
        # delete less freq words:
        delete_words = []
        for word, v in self.word_freq_in_corpus.items():
            if v < self.config["min_word_freq_threshold"]:
                delete_words.append(word)     
        for word in delete_words:
            del self.IDF[word]    
            del self.word_freq_in_corpus[word]    
        
        # delete too freq words
        print('eliminate freq words')
        IDF = [(word, freq) for word, freq in self.IDF.items()]
        IDF.sort(key=lambda x: x[1])

        for i in range(self.config["topk_word_freq_threshold"]):
            print(word)
            word = IDF[i][0]
            del self.IDF[word]
            del self.word_freq_in_corpus[word]
        
        # construct word_vectors
        idx = 1
        for word in self.word_freq_in_corpus:
            self.word_vectors.append(self.word2embedding[word])
            self.stoi[word] = idx
            self.itos[idx] = word
            idx += 1
            
    def init_word_weight(self,sentence_list, agg):
        if agg == 'mean':
            self.word_weight = {word: 1 for word in self.IDF.keys()}
        elif agg == 'IDF':
            self.word_weight = self.IDF
        elif agg == 'uniform':
            self.word_weight = {word: np.random.uniform(low=0.0, high=1.0) for word in self.IDF.keys()}
        elif agg == 'gaussian':
            mu, sigma = 10, 1 # mean and standard deviation
            self.word_weight = {word: np.random.normal(mu, sigma) for word in self.IDF.keys()}
        elif agg == 'exponential':
            self.word_weight = {word: np.random.exponential(scale=1.0) for word in self.IDF.keys()}
        elif agg == 'pmi':
            trigram_measures = BigramAssocMeasures()
            self.word_weight = defaultdict(int)
            corpus = []

            for text in tqdm(sentence_list):
                corpus.extend(text.split())

            finder = BigramCollocationFinder.from_words(corpus)
            for pmi_score in finder.score_ngrams(trigram_measures.pmi):
                pair, score = pmi_score
                self.word_weight[pair[0]] += score
                self.word_weight[pair[1]] += score
                
    def calculate_document_vector(self):
        # Return
        # document_vectors: weighted sum of word emb
        # document_answers_idx: doc to word index list
        # document_answers_wsum: word weight summation, e.g. total TFIDF score of a doc
        
        document_vectors = [] 
        document_answers = []
        document_answers_wsum = []
        
        sentence_list = self.raw_documents
        agg = self.config["document_vector_agg_weight"]
        n_document = self.config["n_document"]
        select_topk_TFIDF = self.config["select_topk_TFIDF"]
        
        self.init_word_weight(sentence_list, agg)
        for sentence in tqdm(sentence_list[:min(n_document, len(sentence_list))], desc="calculate document vectors"):
            document_vector = np.zeros(len(self.word_vectors[0]))
            select_words = []
            for word in self.tokenizer_eng(sentence):
                # pass unknown word
                if word not in self.stoi:
                    continue
                else:
                    select_words.append(word)

            # select topk TDIDF
            if select_topk_TFIDF is not None:
                doc_TFIDF = defaultdict(float)
                for word in select_words:    
                    doc_TFIDF[word] += self.IDF[word]

                doc_TFIDF_l = [(word, TFIDF) for word, TFIDF in doc_TFIDF.items()]
                doc_TFIDF_l.sort(key=lambda x:x[1], reverse=True)
                
                select_topk_words = set(list(map(lambda x:x[0], doc_TFIDF_l[:select_topk_TFIDF])))
                select_words = [word for word in select_words if word in select_topk_words]
            else:
                pass
            
            total_weight = 0
            # aggregate to doc vectors
            for word in select_words:
                document_vector += np.array(self.word2embedding[word]) * self.word_weight[word]
                total_weight += self.word_weight[word]
                
            if len(select_words) == 0:
                print('error', sentence)
                continue
            else:
                if self.config["document_vector_weight_normalize"]:
                    document_vector /= total_weight
                    total_weight = 1
            
            document_vectors.append(document_vector)
            document_answers.append(select_words)
            document_answers_wsum.append(total_weight)
        
        # get answers
        document_answers_idx = []    
        for ans in document_answers:
            ans_idx = []
            for token in ans:
#                 if token in self.stoi:
#                     ans_idx.append(self.stoi[token])     
                ans_idx.append(self.stoi[token])                    
            document_answers_idx.append(ans_idx)
        
        self.document_vectors = document_vectors
        self.document_answers_idx = document_answers_idx
        self.document_answers_wsum = document_answers_wsum
        
        return document_vectors, document_answers_idx, document_answers_wsum
        
    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]
    
    def check_docemb(self):
        word_vectors = np.array(self.word_vectors)
        pred = np.zeros(word_vectors.shape[1])
        cnt = 0

        for word_idx in self.document_answers_idx[0]:
            pred += word_vectors[word_idx] * self.word_weight[self.itos[word_idx]]
            cnt += self.word_weight[self.itos[word_idx]]
        
        if self.config["document_vector_weight_normalize"]:
            pred /= cnt
        assert np.sum(self.document_vectors[0]) - np.sum(pred) == 0

In [7]:
def build_vocab(config, word2embedding):
    # build vocabulary
    vocab = Vocabulary(word2embedding, config)
    vocab.read_raw()
    vocab.build_vocabulary()
    vocab_size = len(vocab)
    # get doc emb
    vocab.calculate_document_vector()
    vocab.check_docemb()
    
    return vocab

vocab = build_vocab(config, word2embedding)

Loading documents: 0it [00:00, ?it/s]

Preprocessing documents:   0%|          | 0/19026 [00:00<?, ?it/s]

doc num 19026
eliminate freq words
hemicrania
subject
line
organ
write
articl
univers
one
would
use
like
get
know
dont
think
time
distribut
make
also
say
peopl
go
im
could
want
new
work
good
well
way
need
look
even
anyon
thing
see
tri
thank
much
comput
year
world
system
right
problem
may
take
mani
two
first
seem
question
pleas
1
state
replyto
us
come
2
post
realli
help
call
usa
point
someth
sinc
find
read
still
believ
back
mean
ive
give
email
sure
differ
might
run
cant
reason
last
day
interest
case
let
person
said
someon
never
anoth
start
doesnt
tell
better
ask
got
without
follow


calculate document vectors:   0%|          | 0/500 [00:00<?, ?it/s]

In [8]:
print("Finish building dataset!")
print(f"Number of documents:{len(vocab.raw_documents)}")
print(f"Number of words:{len(vocab)}")

l = list(map(len, vocab.document_answers_idx))
print("Average length of document:", np.mean(l))

Finish building dataset!
Number of documents:19026
Number of words:8424
Average length of document: 86.922


In [9]:
word_vectors = np.array(vocab.word_vectors)
print("word_vectors:", word_vectors.shape)

document_vectors = np.array(vocab.document_vectors)
print("document_vectors", document_vectors.shape)

document_answers_wsum = np.array(vocab.document_answers_wsum).reshape(-1, 1)
print("document_answers_wsum", document_answers_wsum.shape)

# create weight_ans
document_answers_idx = vocab.document_answers_idx

# random shuffle
# shuffle_idx = list(range(len(document_vectors)))
# random.Random(seed).shuffle(shuffle_idx)

# document_vectors = document_vectors[shuffle_idx]
# document_answers_wsum = document_answers_wsum[shuffle_idx]
# document_answers_idx = [document_answers_idx[idx] for idx in shuffle_idx]

word_vectors: (8424, 100)
document_vectors (500, 100)
document_answers_wsum (500, 1)


In [10]:
# onthot_ans: word freq matrix
# weight_ans: TFIDF matrix

onehot_ans = np.zeros((len(document_answers_idx), word_vectors.shape[0]))
weight_ans = np.zeros((len(document_answers_idx), word_vectors.shape[0]))
print(weight_ans.shape)

for i in tqdm(range(len(document_answers_idx))):
    for word_idx in document_answers_idx[i]:
        weight_ans[i, word_idx] += vocab.word_weight[vocab.itos[word_idx]]
        onehot_ans[i, word_idx] += 1
        
    if config["document_vector_weight_normalize"]:
        weight_ans[i] /= np.sum(weight_ans[i])

(500, 8424)


  0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
# check
assert np.sum(document_vectors - np.dot(weight_ans, word_vectors) > 1e-10) == 0

## Results

In [12]:
final_results = []
select_columns = ['model']
for topk in config["topk"]:
    select_columns.append('percision@{}'.format(topk))
for topk in config["topk"]:
    select_columns.append('recall@{}'.format(topk))
for topk in config["topk"]:
    select_columns.append('F1@{}'.format(topk))
for topk in config["topk"]:
    select_columns.append('ndcg@{}'.format(topk))
select_columns.append('ndcg@all')
select_columns

['model',
 'percision@30',
 'percision@40',
 'percision@50',
 'percision@60',
 'percision@70',
 'recall@30',
 'recall@40',
 'recall@50',
 'recall@60',
 'recall@70',
 'F1@30',
 'F1@40',
 'F1@50',
 'F1@60',
 'F1@70',
 'ndcg@30',
 'ndcg@40',
 'ndcg@50',
 'ndcg@60',
 'ndcg@70',
 'ndcg@all']

## setting training size

In [13]:
train_size_ratio = 1
train_size = int(len(document_answers_idx) * train_size_ratio)
train_size

500

## Top K freq word

In [14]:
topk_results = {}

In [15]:
test_ans = document_answers_idx[:train_size]

In [16]:
word_freq = [(word, freq) for word, freq in vocab.word_freq_in_corpus.items()]
word_freq.sort(key=lambda x:x[1], reverse=True)
word_freq[:10]

[('x', 6539),
 ('god', 5208),
 ('file', 4918),
 ('0', 4520),
 ('window', 4444),
 ('3', 4263),
 ('program', 4201),
 ('drive', 3633),
 ('number', 3611),
 ('4', 3528)]

In [17]:
def topk_word_evaluation(k=50):
    topk_word = [word for (word, freq) in word_freq[:k]]

    pr, re = [], []
    for ans in tqdm(test_ans):
        ans = set(ans)
        ans = [vocab.itos[a] for a in ans]

        hit = []
        for word in ans:
            if word in topk_word:
                hit.append(word)

        precision = len(hit) / k
        recall = len(hit) / len(ans)
        pr.append(precision)
        re.append(recall)

    pr = np.mean(pr)
    re = np.mean(re)
    f1 = 2 * pr * re / (pr + re) if (pr + re) != 0 else 0
    print('top {} word'.format(k))
    print('percision', np.mean(pr))
    print('recall', np.mean(re))
    print('F1', f1)
    return f1


for topk in config['topk']:
    topk_results["F1@{}".format(topk)] = topk_word_evaluation(k=topk)


  0%|          | 0/500 [00:00<?, ?it/s]

top 30 word
percision 0.07719999999999999
recall 0.042731690354719985
F1 0.05501275743929432


  0%|          | 0/500 [00:00<?, ?it/s]

top 40 word
percision 0.07995000000000002
recall 0.05816490823700157
F1 0.0673393549314533


  0%|          | 0/500 [00:00<?, ?it/s]

top 50 word
percision 0.07840000000000001
recall 0.06978781855353748
F1 0.07384365365525153


  0%|          | 0/500 [00:00<?, ?it/s]

top 60 word
percision 0.07933333333333333
recall 0.08367141124931968
F1 0.0814446472230981


  0%|          | 0/500 [00:00<?, ?it/s]

top 70 word
percision 0.07834285714285714
recall 0.09719421048109488
F1 0.08675628743150099


In [18]:
def topk_word_evaluation_NDCG(k=50):
    freq_word =[word for (word, freq) in word_freq]
    freq_word_idx = [vocab.stoi[word] for word in freq_word if word in vocab.stoi]
    
    scores = np.zeros(len(vocab.word_vectors))
    for rank, idx in enumerate(freq_word_idx):
        scores[idx] = len(vocab.word_vectors) - rank
    
    NDCGs = []
    
    for ans in tqdm(test_ans):
        weight_ans = np.zeros(len(vocab.word_vectors))
        
        for word_idx in ans:
            if word_idx == 0:
                continue
            word = vocab.itos[word_idx]
            weight_ans[word_idx] += vocab.IDF[word]

        NDCG_score = ndcg_score(weight_ans.reshape(1,-1), scores.reshape(1,-1), k=k)
        NDCGs.append(NDCG_score)

    print('top {} NDCG:{}'.format(k, np.mean(NDCGs)))
    
    return np.mean(NDCGs)


# for topk in config['topk']:
#     topk_results["ndcg@{}".format(topk)] = topk_word_evaluation_NDCG(k=topk)
    
# topk_results["ndcg@all"] = topk_word_evaluation_NDCG(k=None)


In [19]:
topk_results["model"] = "topk"
final_results.append(pd.Series(topk_results))

## Sklearn

In [20]:
from sklearn.linear_model import LinearRegression, Lasso

In [21]:
print(document_vectors.shape)
print(weight_ans.shape)
print(word_vectors.shape)

(500, 100)
(500, 8424)
(8424, 100)


In [22]:
# def evaluate_sklearn(pred, ans):
#     results = {}
        
#     one_hot_ans = np.arange(ans.shape[0])[ans > 0]
    
#     for topk in config["topk"]:
#         one_hot_pred = np.argsort(pred)[-topk:]
#         hit = np.intersect1d(one_hot_pred, one_hot_ans)
#         percision = len(hit) / topk
#         recall = len(hit) / len(one_hot_ans)
#         f1 = 2 * percision * recall / (percision + recall) if (percision + recall) > 0 else 0
        
#         results['percision@{}'.format(topk)] = percision
#         results['recall@{}'.format(topk)] = recall
#         results['F1@{}'.format(topk)] = f1
        
#     ans = ans.reshape(1, -1)
#     pred = pred.reshape(1, -1)
#     for topk in config["topk"]:
#         results['ndcg@{}'.format(topk)] = ndcg_score(ans, pred, k=topk)

#     results['ndcg@all'] = (ndcg_score(ans, pred, k=None))
    
#     return results

In [23]:
def evaluate_sklearn(pred, ans):
    results = {}
        
    # one_hot_ans = np.arange(ans.shape[0])[ans > 0]
    
    for topk in config["topk"]:
        one_hot_pred = np.argsort(pred)[-topk:]
        one_hot_ans = np.argsort(ans)[-topk:]
        hit = np.intersect1d(one_hot_pred, one_hot_ans)
        percision = len(hit) / topk
        recall = len(hit) / len(one_hot_ans)
        f1 = 2 * percision * recall / (percision + recall) if (percision + recall) > 0 else 0
        
        results['percision@{}'.format(topk)] = percision
        results['recall@{}'.format(topk)] = recall
        results['F1@{}'.format(topk)] = f1
        
    ans = ans.reshape(1, -1)
    pred = pred.reshape(1, -1)
    for topk in config["topk"]:
        results['ndcg@{}'.format(topk)] = ndcg_score(ans, pred, k=topk)

    results['ndcg@all'] = (ndcg_score(ans, pred, k=None))
    
    return results

### linear regression

In [24]:
results = []

for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
    x = word_vectors.T
    y = doc_emb
    
    ans = weight_ans[doc_id]
    model = LinearRegression(fit_intercept=False).fit(x, y)
    r2 = model.score(x, y)

    res = evaluate_sklearn(model.coef_, ans)
    results.append(res)

  0%|          | 0/500 [00:00<?, ?it/s]

In [25]:
results = pd.DataFrame(results).mean()
results['model'] = 'sk-linear-regression'
final_results.append(results)
results

percision@30                0.108133
recall@30                   0.108133
F1@30                       0.108133
percision@40                 0.10075
recall@40                    0.10075
F1@40                        0.10075
percision@50                 0.09508
recall@50                    0.09508
F1@50                        0.09508
percision@60               0.0910667
recall@60                  0.0910667
F1@60                      0.0910667
percision@70               0.0875429
recall@70                  0.0875429
F1@70                      0.0875429
ndcg@30                     0.233143
ndcg@40                      0.23166
ndcg@50                     0.233064
ndcg@60                     0.235806
ndcg@70                     0.239341
ndcg@all                    0.480394
model           sk-linear-regression
dtype: object

### lasso

In [26]:
results = []
sk_lasso_epoch = 2000

for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
    x = word_vectors.T
    y = doc_emb
    
    ans = weight_ans[doc_id]
    model = Lasso(positive=True, fit_intercept=False, alpha=0.0000001, max_iter=sk_lasso_epoch, tol=0).fit(x, y)
    r2 = model.score(x, y)

    res = evaluate_sklearn(model.coef_, ans)
    results.append(res)

  0%|          | 0/500 [00:00<?, ?it/s]

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.037088572391e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.340896593173934e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.509691323828917e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase 

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.560104600516477e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.919916207822555e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.0495689348260475e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to incre

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.615312673351825e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.455947493133514e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.874272144729129e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increa

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.047921374787289e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.9886352127813475e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.5365162749124234e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to incr

...

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.434629889594304e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.280003798890414e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0133407895249542e-05, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to incre

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.370288201786034e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.09800332477456e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.666393660556351e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increas

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.639438839286708e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.443130334883641e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.964426882778494e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increa

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.124007641903237e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0659514755181826e-05, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.038082067462052e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to incre

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.643757840296977e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0037920851975433e-05, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.108990412706945e-06, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to incre

In [27]:
results = pd.DataFrame(results).mean()
results['model'] = 'sk-lasso'
final_results.append(results)
results

percision@30      0.2156
recall@30         0.2156
F1@30             0.2156
percision@40      0.1953
recall@40         0.1953
F1@40             0.1953
percision@50       0.177
recall@50          0.177
F1@50              0.177
percision@60    0.164233
recall@60       0.164233
F1@60           0.164233
percision@70    0.153743
recall@70       0.153743
F1@70           0.153743
ndcg@30         0.444816
ndcg@40         0.440609
ndcg@50         0.439531
ndcg@60         0.440939
ndcg@70         0.443445
ndcg@all        0.614529
model           sk-lasso
dtype: object

## Basic Pursuit

### OMP

In [28]:
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV

results = []

for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
    x = word_vectors.T
    y = doc_emb
    
    ans = weight_ans[doc_id]
    n_nonzero_coefs = np.sum(weight_ans[doc_id] > 0) // 1
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs, tol=None, normalize=True, fit_intercept=False)
    omp.fit(x, y)

    res = evaluate_sklearn(omp.coef_, ans)
    results.append(res)

  0%|          | 0/500 [00:00<?, ?it/s]

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_X=copy_X, return_path=return_path)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_X=copy_X, return_path=return_path)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_X=copy_X, return_path=return_path)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_X=copy_X, return_path=return_path)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_X=copy_X, return_path=return_path)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_X=copy_X, return_path=return_path)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_X=copy_X, return_path=return_path)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_X=copy_X, return_path=return_path)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_X=copy_X, return_path=return_path)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_omp.py:390: RuntimeWarning:  Orthogonal

In [29]:
results = pd.DataFrame(results).mean()
results['model'] = 'sk-omp'
final_results.append(results)
results

percision@30       0.0718
recall@30          0.0718
F1@30              0.0718
percision@40       0.1252
recall@40          0.1252
F1@40              0.1252
percision@50      0.16312
recall@50         0.16312
F1@50             0.16312
percision@60       0.1991
recall@60          0.1991
F1@60              0.1991
percision@70     0.256629
recall@70        0.256629
F1@70            0.256629
ndcg@30         0.0803833
ndcg@40         0.0778496
ndcg@50         0.0765623
ndcg@60         0.0760423
ndcg@70         0.0760789
ndcg@all         0.317385
model              sk-omp
dtype: object

### BPDN

In [30]:
import pylops

results = []

for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
    d = doc_emb
    
    ans = weight_ans[doc_id]
    xspgl1, pspgl1, info = \
        pylops.optimization.sparsity.SPGL1(word_vectors.T, d, None, tau=0, sigma=1e-3, iter_lim=400)

    res = evaluate_sklearn(xspgl1, ans)
    results.append(res)

  0%|          | 0/500 [00:00<?, ?it/s]

In [31]:
results = pd.DataFrame(results).mean()
results['model'] = 'BPDN'
final_results.append(results)
results

percision@30     0.108267
recall@30        0.108267
F1@30            0.108267
percision@40       0.0972
recall@40          0.0972
F1@40              0.0972
percision@50      0.08956
recall@50         0.08956
F1@50             0.08956
percision@60    0.0839667
recall@60       0.0839667
F1@60           0.0839667
percision@70    0.0789714
recall@70       0.0789714
F1@70           0.0789714
ndcg@30          0.253592
ndcg@40          0.248962
ndcg@50          0.248389
ndcg@60          0.249506
ndcg@70          0.251562
ndcg@all         0.473409
model                BPDN
dtype: object

## Our Model

In [32]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [33]:
class Custom_Lasso_Dataset(Dataset):
    def __init__(self, 
                 doc_vectors,
                 doc_w_sum,
                 weight_ans
                 ):
        self.doc_vectors = torch.FloatTensor(doc_vectors)
        self.doc_w_sum = torch.FloatTensor(doc_w_sum)
        self.weight_ans = weight_ans
        assert len(doc_vectors) == len(doc_w_sum)
        
    def __getitem__(self, idx):
                
        return self.doc_vectors[idx], self.doc_w_sum[idx], idx

    def __len__(self):
        return len(self.doc_vectors)


In [34]:
class LR(nn.Module):
    """
    Input shape: (N, 3, 64, 64)
    Output shape: (N, )
    """
    def __init__(self, num_doc, num_words):
        super(LR, self).__init__()
        weight = torch.zeros(num_doc, num_words).to(device)
        self.emb = torch.nn.Embedding.from_pretrained(weight, freeze=False)
        
    def forward(self, doc_ids, word_vectors):
        return self.emb(doc_ids) @ word_vectors

In [35]:
# def evaluate_Custom_Lasso(model, train_loader):
#     results = {}
#     model.eval()
    
#     scores = np.array(model.emb.cpu().weight.data)
#     model.emb.to(device)
#     true_relevance = train_loader.dataset.weight_ans

#     # F1
#     F1s = []
#     precisions = []
#     recalls = []
#     for i in range(true_relevance.shape[0]):
#         one_hot_ans = np.arange(true_relevance.shape[1])[true_relevance[i] > 0]
#         pred = scores[i]
        
#         F1_ = []
#         percision_ = []
#         recall_ = []
#         for topk in config["topk"]:
#             one_hot_pred = np.argsort(pred)[-topk:]
            
#             hit = np.intersect1d(one_hot_pred, one_hot_ans)
#             percision = len(hit) / topk
#             recall = len(hit) / len(one_hot_ans)
            
#             F1 = 2 * percision * recall / (percision + recall) if (percision + recall) > 0 else 0
#             F1_.append(F1)
#             percision_.append(percision)
#             recall_.append(recall)
            
#         F1s.append(F1_)
#         precisions.append(percision_)
#         recalls.append(recall_)
        
#     F1s = np.mean(F1s, axis=0)
#     precisions = np.mean(precisions, axis=0)
#     recalls = np.mean(recalls, axis=0)
    
#     for i, topk in enumerate(config["topk"]):
#         results['F1@{}'.format(topk)] = F1s[i]
#         results['percision@{}'.format(topk)] = precisions[i]
#         results['recall@{}'.format(topk)] = recalls[i]

#     # NDCG
#     for topk in config["topk"]:
#         results['ndcg@{}'.format(topk)] = ndcg_score(true_relevance, scores, k=topk)
#     results['ndcg@all'] = ndcg_score(true_relevance, scores, k=None)
    
#     return results

In [36]:
def evaluate_Custom_Lasso(model, train_loader):
    results = {}
    model.eval()
    
    scores = np.array(model.emb.cpu().weight.data)
    model.emb.to(device)
    true_relevance = train_loader.dataset.weight_ans

    # F1
    F1s = []
    precisions = []
    recalls = []
    for i in range(true_relevance.shape[0]):
        one_hot_ans = np.arange(true_relevance.shape[1])[true_relevance[i] > 0]
        pred = scores[i]
        
        F1_ = []
        percision_ = []
        recall_ = []
        for topk in config["topk"]:
            topk = min(topk, sum(true_relevance[i] > 0))
            one_hot_pred = np.argsort(pred)[-topk:]
            one_hot_ans = np.argsort(true_relevance[i])[-topk:]
            
            hit = np.intersect1d(one_hot_pred, one_hot_ans)
            percision = len(hit) / topk
            recall = len(hit) / len(one_hot_ans)
            
            F1 = 2 * percision * recall / (percision + recall) if (percision + recall) > 0 else 0
            F1_.append(F1)
            percision_.append(percision)
            recall_.append(recall)
            
        F1s.append(F1_)
        precisions.append(percision_)
        recalls.append(recall_)
        
    F1s = np.mean(F1s, axis=0)
    precisions = np.mean(precisions, axis=0)
    recalls = np.mean(recalls, axis=0)
    
    for i, topk in enumerate(config["topk"]):
        results['F1@{}'.format(topk)] = F1s[i]
        results['percision@{}'.format(topk)] = precisions[i]
        results['recall@{}'.format(topk)] = recalls[i]

    # NDCG
    for topk in config["topk"]:
        results['ndcg@{}'.format(topk)] = ndcg_score(true_relevance, scores, k=topk)
    results['ndcg@all'] = ndcg_score(true_relevance, scores, k=None)
    
    return results

In [37]:
batch_size = 100
print('document num', train_size)

train_dataset = Custom_Lasso_Dataset(document_vectors[:train_size], document_answers_wsum[:train_size], weight_ans[:train_size])
train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

document num 500


## start training

In [38]:
def train_CLasso(select_idx):
    batch_size = len(select_idx)
    print('batch_size', batch_size)
    train_dataset = Custom_Lasso_Dataset(document_vectors[select_idx], document_answers_wsum[select_idx], weight_ans[select_idx])
    train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    
    model = LR(num_doc=batch_size, num_words=word_vectors.shape[0]).to(device)
    model.train()

    word_vectors_tensor = torch.FloatTensor(word_vectors).to(device)

    opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
    criterion = nn.MSELoss(reduction='none')

    results = []
    step = 0
    for epoch in tqdm(range(n_epoch)):    
        loss_mse_his = []
        loss_w_reg_his = []

        model.train()

        for data in train_loader:
            doc_embs, doc_w_sum, doc_ids = data

            doc_embs = doc_embs.to(device)
            doc_w_sum = doc_w_sum.to(device)
            doc_ids = doc_ids.to(device)

            w_reg = doc_w_sum * w_sum_reg_mul
            # w_reg = (torch.ones(doc_embs.size(0), 1) * w_sum_reg_mul).to(device)

            # MSE loss
            pred_doc_embs = model(doc_ids, word_vectors_tensor)     
            loss_mse = criterion(pred_doc_embs, doc_embs)
            loss_mse = torch.sum(torch.mean(loss_mse, dim=1))

            pred_w_sum = torch.sum(model.emb(doc_ids), axis=1).view(-1, 1)
            loss_w_reg = criterion(pred_w_sum, w_reg)
            loss_w_reg = torch.sum(torch.mean(loss_w_reg, dim=1))

            loss_l1 = torch.sum(torch.abs(model.emb(doc_ids)))
            loss = loss_mse + loss_w_reg * w_sum_reg + loss_l1 * L1

            # Model backwarding
            model.zero_grad()
            loss.backward()
            opt.step()

            loss_mse_his.append(loss_mse.item())
            loss_w_reg_his.append(loss_w_reg.item())

            for p in model.parameters():
                p.data.clamp_(w_clip_value, float('inf'))
#                 p.data -= L1
#                 p.data[p.data < 0] = 0


        if (verbose and epoch % valid_epoch == 0) or (not verbose and epoch == n_epoch-1):
            res = {}
            res['epoch'] = epoch
            res['loss_mse'] = np.mean(loss_mse_his)
            res['loss_w_reg'] = np.mean(loss_w_reg_his)

            res_ndcg = evaluate_Custom_Lasso(model, train_loader)
            res.update(res_ndcg)
            results.append(res)

            print()
            for k, v in res.items():
                print(k, v)
                
    return res, model

# setting
lr = 0.03
momentum = 0.9995
weight_decay = 0#1e-6
nesterov = False # True

n_epoch = 2000

w_sum_reg = 1e-2
w_sum_reg_mul = 1
w_clip_value = 0

L1 = 1e-5

verbose = False
valid_epoch = 100

results = []
for i in range(len(document_vectors)//batch_size+1):
    if i != len(document_vectors)//batch_size:
        select_idx = np.arange(i*batch_size, (i+1)*batch_size)
    elif len(document_vectors)%batch_size != 0:
        select_idx = np.arange(i*batch_size, len(document_vectors))
    else:
        break
    res, model = train_CLasso(select_idx)
    results.append(res)

batch_size 100


  0%|          | 0/2000 [00:00<?, ?it/s]


epoch 1999
loss_mse 0.015121258795261383
loss_w_reg 5.408492088317871
F1@30 0.2866937529817521
percision@30 0.2866937529817521
recall@30 0.2866937529817521
F1@40 0.275452333155867
percision@40 0.275452333155867
recall@40 0.275452333155867
F1@50 0.2710201764774311
percision@50 0.27102017647743104
recall@50 0.27102017647743104
F1@60 0.2702608344320209
percision@60 0.2702608344320209
recall@60 0.2702608344320209
F1@70 0.2683567791692903
percision@70 0.2683567791692903
recall@70 0.2683567791692903
ndcg@30 0.5273084263484809
ndcg@40 0.5120865027138994
ndcg@50 0.5072513362527029
ndcg@60 0.5066924966821982
ndcg@70 0.507242133526913
ndcg@all 0.6890259837876279
batch_size 100


  0%|          | 0/2000 [00:00<?, ?it/s]


epoch 1999
loss_mse 0.018449366092681885
loss_w_reg 7.710880756378174
F1@30 0.32463694603674087
percision@30 0.32463694603674087
recall@30 0.32463694603674087
F1@40 0.31302500560020363
percision@40 0.31302500560020363
recall@40 0.31302500560020363
F1@50 0.3032435819145115
percision@50 0.3032435819145115
recall@50 0.3032435819145115
F1@60 0.29769209279069025
percision@60 0.29769209279069025
recall@60 0.29769209279069025
F1@70 0.2963216614123266
percision@70 0.2963216614123266
recall@70 0.2963216614123266
ndcg@30 0.5805417457231274
ndcg@40 0.5671085272503473
ndcg@50 0.5600998445936767
ndcg@60 0.558273895971324
ndcg@70 0.5587601716160483
ndcg@all 0.7211485462403124
batch_size 100


...


epoch 1999
loss_mse 0.00899134948849678
loss_w_reg 3.512566566467285
F1@30 0.353408709091219
percision@30 0.353408709091219
recall@30 0.353408709091219
F1@40 0.3448883848799179
percision@40 0.3448883848799179
recall@40 0.3448883848799179
F1@50 0.3381099312352724
percision@50 0.3381099312352724
recall@50 0.3381099312352724
F1@60 0.3319374511105355
percision@60 0.3319374511105355
recall@60 0.3319374511105355
F1@70 0.3310613229347899
percision@70 0.3310613229347899
recall@70 0.3310613229347899
ndcg@30 0.5953630573619325
ndcg@40 0.5861379088597409
ndcg@50 0.5807356170487892
ndcg@60 0.5783378346288994
ndcg@70 0.5792956437814738
ndcg@all 0.7356902365996056
batch_size 100


  0%|          | 0/2000 [00:00<?, ?it/s]


epoch 1999
loss_mse 0.026054926216602325
loss_w_reg 14.213451385498047
F1@30 0.32828649155275846
percision@30 0.32828649155275846
recall@30 0.32828649155275846
F1@40 0.3128259537841398
percision@40 0.3128259537841398
recall@40 0.3128259537841398
F1@50 0.30576841791520026
percision@50 0.30576841791520026
recall@50 0.30576841791520026
F1@60 0.30063008820151293
percision@60 0.300630088201513
recall@60 0.300630088201513
F1@70 0.29824084323481403
percision@70 0.29824084323481403
recall@70 0.29824084323481403
ndcg@30 0.5589912826821659
ndcg@40 0.5433909899138889
ndcg@50 0.5389986141154901
ndcg@60 0.5364952153497076
ndcg@70 0.5362002496832605
ndcg@all 0.7059053540075921
batch_size 100


  0%|          | 0/2000 [00:00<?, ?it/s]


epoch 1999
loss_mse 0.04891330748796463
loss_w_reg 18.705366134643555
F1@30 0.3140425251368956
percision@30 0.3140425251368956
recall@30 0.3140425251368956
F1@40 0.30239725807938933
percision@40 0.30239725807938933
recall@40 0.30239725807938933
F1@50 0.2941644192067486
percision@50 0.2941644192067486
recall@50 0.2941644192067486
F1@60 0.29064175084993454
percision@60 0.29064175084993454
recall@60 0.29064175084993454
F1@70 0.2882759896289214
percision@70 0.2882759896289214
recall@70 0.2882759896289214
ndcg@30 0.56331135466169
ndcg@40 0.5469693925355358
ndcg@50 0.5391215714732076
ndcg@60 0.5390484345221155
ndcg@70 0.5399221349341504
ndcg@all 0.7099703847687315


In [39]:
pd.set_option('display.max_rows', 500)
results_df = pd.DataFrame(results).set_index('epoch').mean()
results_df

loss_mse        0.023506
loss_w_reg      9.910151
F1@30           0.321414
percision@30    0.321414
recall@30       0.321414
F1@40           0.309718
percision@40    0.309718
recall@40       0.309718
F1@50           0.302461
percision@50    0.302461
recall@50       0.302461
F1@60           0.298232
percision@60    0.298232
recall@60       0.298232
F1@70           0.296451
percision@70    0.296451
recall@70       0.296451
ndcg@30         0.565103
ndcg@40         0.551139
ndcg@50         0.545241
ndcg@60         0.543770
ndcg@70         0.544284
ndcg@all        0.712348
dtype: float64

In [40]:
results_df['model'] = 'our-lasso'
final_results.append(results_df[select_columns])

## Quality Check

In [41]:
# select doc_id and k
doc_id = 36
topk = 30

model

LR(
  (emb): Embedding(100, 8424)
)

In [42]:
import colored
from colored import stylize

word_list = vocab.itos

gt = [word_list[word_idx] for word_idx in np.argsort(weight_ans[doc_id])[::-1][:topk]]
pred = [word_list[word_idx] for word_idx in np.argsort(model.emb.cpu().weight.data[doc_id].numpy())[::-1][:topk]]

print('ground truth')
for word in gt:
    if word in pred:
        print(stylize(word, colored.bg("yellow")), end=' ')
    else:
        print(word, end=' ')

print()
print('\nprediction')
for word in pred:
    if word in gt:
        print(stylize(word, colored.bg("yellow")), end=' ')
    else:
        print(word, end=' ')


ground truth
eclips upgrad digit perform hade 33mhz pb dartmouth nh hugh moder compon v letter brian hardwar replac disclaim speed test speak deal chip colleg week type 14 care suggest abl 

prediction
borland recommend learn easi simpli workaround drawback suggest buy apr recurs ms 24 imho lot virtu buyer l isa carol via 7 spss rel 29 smart bet retrospect welch p 

In [43]:
# raw document
print()
ps = PorterStemmer()
    
for word in vocab.raw_documents[doc_id].split():
    word_stem = ps.stem(word).lower()

    if word_stem in gt:
        if word_stem in pred:
            print(stylize(word, colored.bg("yellow")), end=' ')
        else:
            print(stylize(word, colored.bg("light_gray")), end=' ')
    else:
        print(word, end=' ')
# print(dataset.documents[doc_id])


From: hades@coos.dartmouth.edu (Brian V. Hughes) Subject: Re: DESI PB upgrade Reply-To: hades@Dartmouth.Edu Organization: Dartmouth College, Hanover, NH Disclaimer: Personally, I really don't care who you think I speak for. Moderator: Rec.Arts.Comics.Info Lines: 14 David_A._Schnider@bmug.org writes: >Does anyone know exactly how Digital Eclipse does their upgrades? Someone was >suggesting to me that some chips may not be able to perform at 33MHz. Is this >true, and if so, how does DESI deal with that? Would you believe that there is a letter in MacWEEK this week from one of the hardware types at Digital Eclipse. He says that they run tests on all of the components to see if they will perform at the upgraded speed. If they do not then DESI replaces them with ones that do. -Hades 

In [44]:
results = {}
   
scores = np.array(model.emb.weight.data)[doc_id].reshape(1, -1)
true_relevance = train_loader.dataset.weight_ans[doc_id].reshape(1, -1)

results['ndcg@50'] = (ndcg_score(true_relevance, scores, k=50))
results['ndcg@100'] = (ndcg_score(true_relevance, scores, k=100))
results['ndcg@200'] = (ndcg_score(true_relevance, scores, k=200))
results['ndcg@all'] = (ndcg_score(true_relevance, scores, k=None))

print('This document ndcg:')
print('ground truth length:', np.sum(weight_ans[doc_id] > 0))
print('NDCG top50', results['ndcg@50'])
print('NDCG top100', results['ndcg@100'])
print('NDCG top200', results['ndcg@200'])
print('NDCG ALL', results['ndcg@all'])


This document ndcg:
ground truth length: 31
NDCG top50 0.015539943272004128
NDCG top100 0.02684489942261207
NDCG top200 0.038924107346115476
NDCG ALL 0.2551761014507471


## Final results

In [45]:
is_notebook = in_notebook()

In [46]:
import time
t = time.localtime()
t = time.strftime("%Y-%m-%d_%H:%M:%S", t)

final_results_df = pd.DataFrame(final_results).reset_index(drop=True)

experiment_dir = './records/dataset-{}-n_document-{}-wdist-{}-filtertopk-{}-dim-{}-time-{}'.format(
                                        config['dataset'],
                                        config['n_document'],
                                        config["document_vector_agg_weight"],
                                        config["topk_word_freq_threshold"],
                                        config["embedding_file"].split('.')[-2][:-1],
                                        t)

print('Saving to directory', experiment_dir)
os.makedirs(experiment_dir, exist_ok=True)

Saving to directory ./records/dataset-CNN-n_document-500-wdist-IDF-filtertopk-100-dim-100-time-2021-11-15_18:13:38


In [47]:
config = {}

config["dataset"] = "CNN" # "IMDB" "CNN", "PubMed"
config["n_document"] = 100
config["normalize_word_embedding"] = False
config["min_word_freq_threshold"] = 20
config["topk_word_freq_threshold"] = 100
config["document_vector_agg_weight"] = 'IDF' # ['mean', 'IDF', 'uniform', 'gaussian', 'exponential', 'pmi']
config["document_vector_weight_normalize"] = True # weighted sum or mean, True for mean, False for sum 
config["select_topk_TFIDF"] = None # ignore
config["embedding_file"] = "../data/glove.6B.100d.txt"
# config["embedding_file"] = "../data/400d.txt"
config["topk"] = [30, 40, 50, 60, 70]

In [48]:
final_results_df.to_csv(os.path.join(experiment_dir, 'result.csv'), index=False)

import json
with open(os.path.join(experiment_dir, 'config.json'), 'w') as json_file:
    json.dump(config, json_file)

In [49]:
for feat in final_results_df.set_index('model').columns:
    plt.bar(final_results_df['model'],
            final_results_df[feat], 
            width=0.5, 
            bottom=None, 
            align='center', 
            color=['lightsteelblue', 
                   'cornflowerblue', 
                   'royalblue', 
                   'navy'])
    plt.title(feat)
    plt.savefig(os.path.join(experiment_dir, '{}.png'.format(feat)))
    plt.clf()
    if is_notebook:
        plt.show()

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  from ipykernel import kernelapp as app


In [50]:
print(final_results_df)
final_results_df

      F1@30     F1@40     F1@50     F1@60     F1@70                 model  \
0  0.055013  0.067339  0.073844  0.081445  0.086756                  topk   
1  0.108133  0.100750  0.095080  0.091067  0.087543  sk-linear-regression   
2  0.215600  0.195300  0.177000  0.164233  0.153743              sk-lasso   
3  0.071800  0.125200  0.163120  0.199100  0.256629                sk-omp   
4  0.108267  0.097200  0.089560  0.083967  0.078971                  BPDN   
5  0.321414  0.309718  0.302461  0.298232  0.296451             our-lasso   

   percision@30  recall@30  percision@40  recall@40  ...  percision@60  \
0           NaN        NaN           NaN        NaN  ...           NaN   
1      0.108133   0.108133      0.100750   0.100750  ...      0.091067   
2      0.215600   0.215600      0.195300   0.195300  ...      0.164233   
3      0.071800   0.071800      0.125200   0.125200  ...      0.199100   
4      0.108267   0.108267      0.097200   0.097200  ...      0.083967   
5      0.321414 

,F1@30,F1@40,F1@50,F1@60,F1@70,model,percision@30,recall@30,percision@40,recall@40,...,percision@60,recall@60,percision@70,recall@70,ndcg@30,ndcg@40,ndcg@50,ndcg@60,ndcg@70,ndcg@all
0,0.055013,0.067339,0.073844,0.081445,0.086756,topk,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.108133,0.100750,0.095080,0.091067,0.087543,sk-linear-regression,0.108133,0.108133,0.100750,0.100750,...,0.091067,0.091067,0.087543,0.087543,0.233143,0.231660,0.233064,0.235806,0.239341,0.480394
2,0.215600,0.195300,0.177000,0.164233,0.153743,sk-lasso,0.215600,0.215600,0.195300,0.195300,...,0.164233,0.164233,0.153743,0.153743,0.444816,0.440609,0.439531,0.440939,0.443445,0.614529
3,0.071800,0.125200,0.163120,0.199100,0.256629,sk-omp,0.071800,0.071800,0.125200,0.125200,...,0.199100,0.199100,0.256629,0.256629,0.080383,0.077850,0.076562,0.076042,0.076079,0.317385
4,0.108267,0.097200,0.089560,0.083967,0.078971,BPDN,0.108267,0.108267,0.097200,0.097200,...,0.083967,0.083967,0.078971,0.078971,0.253592,0.248962,0.248389,0.249506,0.251562,0.473409
5,0.321414,0.309718,0.302461,0.298232,0.296451,our-lasso,0.321414,0.321414,0.309718,0.309718,...,0.298232,0.298232,0.296451,0.296451,0.565103,0.551139,0.545241,0.543770,0.544284,0.712348


## Basic pursuit

In [51]:
# print(word_vectors.shape)
# print(document_vectors.shape)

In [52]:
# import spgl1

# results = []
# sigma = 1e-3

# for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
#     d = doc_emb
#     ans = weight_ans[doc_id]
#     coef, _, _, info = spgl1.spg_bpdn(A=word_vectors.T, b=d, sigma=sigma)

#     res = evaluate_sklearn(coef, ans)
#     results.append(res)

In [53]:
# results = pd.DataFrame(results).mean()
# results['model'] = 'SPG1'
# final_results.append(results)
# results

In [54]:
# import spgl1

# results = []

# for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
#     d = doc_emb
#     ans = weight_ans[doc_id]
#     coef, _, _, info = spgl1.spg_bp(A=word_vectors.T, b=d)

#     res = evaluate_sklearn(coef, ans)
#     results.append(res)
    

In [55]:
# results = pd.DataFrame(results).mean()
# results['model'] = 'SPG1'
# final_results.append(results)
# results